## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace
import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# Import necessary libraries
import os
import json
import chromadb
from dotenv import load_dotenv
from chromadb.utils import embedding_functions

In [3]:
# Setting API variables

OPENAI_API_KEY = "voc-11905559501688654276185689a2c33e68215.70173222"
CHROMA_OPENAI_API_KEY = "voc-11905559501688654276185689a2c33e68215.70173222"
TAVILY_API_KEY = "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"
VOCAREUM_API_BASE = "https://openai.vocareum.com/v1"

In [4]:
### 1. Load and Process Data from Multiple JSON Files

# *** THIS IS THE MAIN FIX ***
# This section now reads all individual JSON files from the 'games' directory.
data = []
data_dir = 'games'

if os.path.exists(data_dir) and os.path.isdir(data_dir):
    for filename in sorted(os.listdir(data_dir)):
        if filename.endswith('.json'):
            filepath = os.path.join(data_dir, filename)
            try:
                with open(filepath, 'r') as f:
                    game_data = json.load(f)
                    data.append(game_data)
            except json.JSONDecodeError:
                print(f"Warning: Could not decode JSON from {filename}. Skipping file.")
            except Exception as e:
                print(f"An error occurred reading {filename}: {e}")
    print(f"Successfully loaded {len(data)} game records from the '{data_dir}' directory.")
else:
    print(f"Error: Directory '{data_dir}' not found. Please ensure your game JSON files are in this directory.")


# Prepare the data by creating descriptive chunks and metadata
documents = []
metadatas = []
ids = []

for i, item in enumerate(data):
    # Create a descriptive string for each game to be used as the document
    document_chunk = (
        f"Name: {item.get('Name', 'N/A')}\n"
        f"Platform: {item.get('Platform', 'N/A')}\n"
        f"Year of Release: {item.get('Year_of_Release', 'N/A')}\n"
        f"Genre: {item.get('Genre', 'N/A')}\n"
        f"Publisher: {item.get('Publisher', 'N/A')}\n"
        f"Description: {item.get('Description', 'N/A')}"
    )
    
    documents.append(document_chunk)
    
    # Create metadata for each document
    metadatas.append({
        "name": item.get('Name', 'N/A'),
        "platform": item.get('Platform', 'N/A'),
        "year": item.get('Year_of_Release', 'N/A'),
        "genre": item.get('Genre', 'N/A'),
        "publisher": item.get('Publisher', 'N/A')
    })
    
    # Create a unique ID for each document
    ids.append(f"game_{i}")

print(f"Processed {len(documents)} documents for the vector database.")

Successfully loaded 15 game records from the 'games' directory.
Processed 15 documents for the vector database.


In [5]:
### 2. Create and Populate the Vector Database
if documents:
    # Initialize the ChromaDB client for persistent storage
    chroma_client = chromadb.PersistentClient(path="chromadb")

    # Define the embedding function with the correct API details
    # This ensures the embeddings are generated correctly for your custom environment
    embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=OPENAI_API_KEY,
        api_base=VOCAREUM_API_BASE,
        model_name="text-embedding-ada-002"
    )

    # Create or get the collection
    collection = chroma_client.get_or_create_collection(
        name="udaplay",
        embedding_function=embedding_fn
    )

    # *** THIS IS THE MAIN FIX ***
    # The reviewer noted a TypeError. The fix is to use the correct keyword argument 'documents'
    # instead of 'document', and to ensure the data is passed as a list.
    # The code now correctly adds all documents, metadatas, and ids in a single batch.
    try:
        collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )
        print("\nSuccessfully added all documents to the 'udaplay' collection in ChromaDB.")
        print(f"Total items in collection: {collection.count()}")

    except Exception as e:
        print(f"\nAn error occurred while adding documents to ChromaDB: {e}")


Successfully added all documents to the 'udaplay' collection in ChromaDB.
Total items in collection: 15


In [6]:
### 3. Demonstrate Semantic Search
if documents and 'collection' in locals():
    print("\n--- Demonstrating Semantic Search ---")
    
    # Define a sample query
    query = "Which games were published by Nintendo on the Wii?"
    
    print(f"\nQuerying the database for: '{query}'")
    
    # Perform the search
    results = collection.query(
        query_texts=[query],
        n_results=5,
        include=['documents', 'metadatas']
    )
    
    # Display the results
    print("\nTop 5 Search Results:")
    for i, (doc, meta) in enumerate(zip(results['documents'][0], results['metadatas'][0])):
        print(f"\nResult {i+1}:")
        print(f"  Game: {meta.get('name', 'N/A')}")
        print(f"  Platform: {meta.get('platform', 'N/A')}")
        print(f"  Year: {meta.get('year', 'N/A')}")
        print(f"  Document Chunk:\n---\n{doc}\n---")
else:
    print("\nSkipping database operations because no data was loaded.")


--- Demonstrating Semantic Search ---

Querying the database for: 'Which games were published by Nintendo on the Wii?'

Top 5 Search Results:

Result 1:
  Game: Wii Sports
  Platform: Wii
  Year: N/A
  Document Chunk:
---
Name: Wii Sports
Platform: Wii
Year of Release: N/A
Genre: Sports
Publisher: Nintendo
Description: A collection of sports games that utilize the Wii's motion controls, bundled with the console to showcase its capabilities.
---

Result 2:
  Game: Super Mario 64
  Platform: Nintendo 64
  Year: N/A
  Document Chunk:
---
Name: Super Mario 64
Platform: Nintendo 64
Year of Release: N/A
Genre: Platformer
Publisher: Nintendo
Description: A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.
---

Result 3:
  Game: Super Mario World
  Platform: Super Nintendo Entertainment System (SNES)
  Year: N/A
  Document Chunk:
---
Name: Super Mario World
Platform: Super Nintendo Entertainment System (SNES)
Year of Release: 